In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from PIL import Image, ImageDraw
from ast import literal_eval
import openslide
import cv2
import re
import json
import tifffile
from scipy.ndimage import zoom

slide_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/training/all/'
slide_mask_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/training_masks/'
lesion_mask_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/lesion_annotations' # .xml files
gt_mask_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/training_masks/'
whole_slide_pred_path = '../non_overlap_tiles/whole_slide_prediction/'

pfmap_path = "/fs/ess/PAS1575/Dataset/new_data/fixation_masks/fixation_reduction/"

output_path = './gt_and_fixation_new_zoom/'

down_scale = 16 # default value that needs to be adjusted

In [41]:
# load in eye fixation data
pids = ['P5']

df_image = pd.read_csv('../experimentData/imageNameMappingWithTumorRatio.csv')

df_eyetrack = pd.read_csv('./TrackerData/Exp1_CombinedGazeMouseData.csv')
df_eyetrack = df_eyetrack[df_eyetrack['participantID'].isin(pids)]

eyetrack_images = df_eyetrack['imageID'].dropna().unique()
print(f"{len(eyetrack_images)} unique images in eye tracking data")

60 unique images in eye tracking data


In [45]:
# Convert to DataFrame for merging
df_test_reference = pd.read_csv('../cam16_test_reference.csv')

df_combined = pd.merge(df_image, df_test_reference, left_on='imageID', right_on='image_id')
df_combined = df_combined[~df_combined['imageID'].str.contains('TC')]
df_combined = df_combined[['imageID', 'imageName', 'type', 'level']]
df_combined.columns = ['img_id', 'image_name', 'type', 'level']

# get participantID and imageID pairs from df_eyetrack
df_eyetrack_pairs = df_eyetrack[['participantID', 'imageID']].dropna().drop_duplicates()
df_eyetrack_pairs.columns = ['pid', 'img_id']

# merge with df_combined to get levels
df_combined = pd.merge(df_combined, df_eyetrack_pairs, on='img_id')

# merge with df_combined to get correctness
df_pid_answers = pd.read_csv('../experimentData/allParticipantsAnswers.csv')
df_pid_answers = df_pid_answers[['pID', 'imageID', 'correctness']]
df_pid_answers.columns = ['pid', 'img_id', 'correctness']
df_combined = pd.merge(df_combined, df_pid_answers, on=['pid', 'img_id'])

# change none to Normal in level and delete type column
df_combined['level'] = df_combined['level'].replace('None', 'Normal')
df_combined = df_combined.drop(columns=['type'])

In [46]:
import numpy as np
import cv2
import openslide
from PIL import Image

def _choose_level_for_downscale(slide, desired_down):
    downs = slide.level_downsamples
    idx = int(np.argmin([abs(d - desired_down) for d in downs]))
    return idx, float(downs[idx])

def _rgba_to_rgb(pil_img):
    if pil_img.mode == 'RGBA':
        return Image.alpha_composite(
            Image.new('RGBA', pil_img.size, (255, 255, 255, 255)),
            pil_img
        ).convert('RGB')
    return pil_img.convert('RGB')

def get_thumbnail_white_bg(
    slide_name,
    gt_color=(255, 0, 0),
    pad_pixels=0,
    roi_bbox_thumb=None    # (xmin_t, ymin_t, xmax_t, ymax_t) in thumbnail px
):
    """
    Reads an ROI (or full slide) using read_region, draws GT contours, 
    and preserves the original coordinate offset.

    Returns:
      thumbnail_np_out       : RGB ndarray of ROI
      thumbnail_with_gt_out  : RGB ndarray with GT contours drawn in red
      global_offset          : (xmin_global, ymin_global) — top-left coord in original slide
      bbox_used_thumb        : [xmin_t, ymin_t, xmax_t, ymax_t] in thumbnail px
    """
    # Build paths from globals
    imgPath = slide_path + slide_name + '.tif'
    imgPathMask = slide_mask_path + slide_name + '_mask.tif'

    # --- Open mask to get full dimensions and pick pyramid level ---
    with openslide.open_slide(imgPathMask) as mask_slide:
        nx, ny = mask_slide.dimensions
        L_idx, L_down = _choose_level_for_downscale(mask_slide, down_scale)

        # If no ROI, default to entire slide at thumbnail scale
        if roi_bbox_thumb is None:
            W_t = int(round(nx / down_scale))
            H_t = int(round(ny / down_scale))
            roi_bbox_thumb = (0, 0, W_t, H_t)

        xmin_t, ymin_t, xmax_t, ymax_t = map(int, roi_bbox_thumb)
        # optional pad in thumbnail units
        xmin_t = max(0, xmin_t - pad_pixels)
        ymin_t = max(0, ymin_t - pad_pixels)
        xmax_t = min(int(round(nx / down_scale)), xmax_t + pad_pixels)
        ymax_t = min(int(round(ny / down_scale)), ymax_t + pad_pixels)

        # Map to level-0 coords and compute ROI size
        xmin0 = int(round(xmin_t * down_scale))
        ymin0 = int(round(ymin_t * down_scale))
        xmax0 = int(round(xmax_t * down_scale))
        ymax0 = int(round(ymax_t * down_scale))
        w0, h0 = xmax0 - xmin0, ymax0 - ymin0

        wL = max(1, int(round(w0 / L_down)))
        hL = max(1, int(round(h0 / L_down)))

        # Read the mask ROI
        mask_rgba = mask_slide.read_region((xmin0, ymin0), L_idx, (wL, hL))
        mask_gray = np.array(_rgba_to_rgb(mask_rgba).convert('L'))

    # --- Read image ROI at same coords/level ---
    with openslide.open_slide(imgPath) as slide:
        region_rgba = slide.read_region((xmin0, ymin0), L_idx, (wL, hL))
    region_rgb = np.array(_rgba_to_rgb(region_rgba))

    # --- Draw GT contours ---
    gt_mask = (mask_gray > 0).astype(np.uint8)
    contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    region_rgb_with_gt = region_rgb.copy()
    if contours:
        roi_bgr = cv2.cvtColor(region_rgb_with_gt, cv2.COLOR_RGB2BGR)
        cv2.drawContours(roi_bgr, contours, -1, gt_color, int(4*32/down_scale))
        region_rgb_with_gt = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2RGB)

    # Return image, overlay, and the global offset (top-left in original coords)
    global_offset = (xmin0, ymin0)
    bbox_used_thumb = [xmin_t, ymin_t, xmax_t, ymax_t]

    return region_rgb, region_rgb_with_gt, global_offset, bbox_used_thumb

In [62]:
# Set image ID and participant ID
img_id = 'C237'
pid = 'P5'

slide_name = df_image.at[df_image.index[df_image['imageID']==img_id].values[0], 'imageName']
print(f"Processing slide: {slide_name}")

# set display extent in 32-downscaled coordinates
xmin, ymin, xmax, ymax = 700*32/down_scale, 3900*32/down_scale, 1800*32/down_scale, 5400*32/down_scale


Processing slide: tumor_016


In [50]:
### Draw lesion annotations on black background ###

import xml.etree.ElementTree as ET

def load_camelyon_asap_polygons(xml_path, down_scale=1.0, tumor_only=True):
    """
    Load lesion polygons from a CAMELYON16 XML file and downscale coordinates.
    Returns a list of Nx2 arrays [[x0, y0], [x1, y1], ...].
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    annotations_el = root.find("Annotations")
    if annotations_el is None:
        return []

    polygons = []

    for annot in annotations_el.findall("Annotation"):
        if tumor_only:
            group = annot.get("PartOfGroup", "")
            if group.lower() not in ("tumor", "tumour"):
                continue

        coords_el = annot.find("Coordinates")
        if coords_el is None:
            continue

        coords = []
        for coord in coords_el.findall("Coordinate"):
            x = float(coord.get("X")) / down_scale
            y = float(coord.get("Y")) / down_scale
            coords.append((x, y))

        if coords:
            polygons.append(np.array(coords, dtype=float))

    return polygons


def draw_lesions_on_black(lesion_mask_file, down_scale, xmin, ymin, xmax, ymax,
                          figsize=(6, 6), linewidth=1.5):
    """
    Draw lesion annotations as red lines on a black background
    in the region [xmin, ymin, xmax, ymax] (downscaled coordinates).
    """

    # 1. Load polygons (already downscaled)
    polys = load_camelyon_asap_polygons(lesion_mask_file, down_scale=down_scale, tumor_only=False)

    # 2. Set up figure with black background
    fig, ax = plt.subplots(figsize=figsize, dpi=300)
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    # 3. Plot each polygon as a red line
    for poly in polys:
        xs = poly[:, 0]
        ys = poly[:, 1]
        ax.plot(xs, ys, color='red', linewidth=linewidth)

    # 4. Restrict to the ROI: [xmin, ymin, xmax, ymax]
    ax.set_xlim(xmin, xmax)
    # In image coordinates y increases downward, so invert Y:
    ax.set_ylim(ymax, ymin)

    plt.tight_layout()
    return fig, ax

In [54]:
# Get mask width and height
gt_path = os.path.join(gt_mask_path, f'{slide_name}_mask.tif')
gt_mask = tifffile.imread(gt_path)

# Display the original mask shape
print(f"Original mask shape: {gt_mask.shape}")

# Shrink the mask by a factor of 224 (downsize)
# Since we are shrinking, we use a zoom factor of 1/224 for both dimensions
shrink_factor = 1 / 224
shrunk_mask = zoom(gt_mask, zoom=shrink_factor, order=0)  # Nearest-neighbor interpolation (order=0)

slide_width = shrunk_mask.shape[1]
slide_height = shrunk_mask.shape[0]
print(slide_width, slide_height)
print('sum of tumor pixels', np.sum(shrunk_mask>1))

Original mask shape: (221184, 97792)
437 987
sum of tumor pixels 3184


In [63]:
### Extract prediction probability and show heatmap ###
df = pd.read_csv(os.path.join(whole_slide_pred_path, f'{slide_name}.csv'))

min_x = df['x'].min()
min_y = df['y'].min()
max_x = df['x'].max()
max_y = df['y'].max()
print(f'{min_x} to {max_x}, {min_y} to {max_y}')

mask = np.zeros((slide_height, slide_width), dtype=np.float32)

# Fill the mask with the probability values from yhat at the given x and y coordinates
for _, row in df.iterrows():
    x, y, yhat = int(row['x']), int(row['y']), row['yhat']
    mask[y, x] = yhat  # y is row index, x is column index

scale = int(224 / down_scale)

upscaled = np.repeat(
    np.repeat(mask, scale, axis=0), 
    scale, axis=1
)

# Display the mask using matplotlib
plt.imshow(upscaled, cmap='hot')

plt.xlim(xmin, xmax)
plt.ylim(ymax, ymin)

plt.colorbar(label='Predicted tumor probability', )
plt.savefig(output_path + f'{slide_name}_expanded_mask.png', bbox_inches='tight') 
plt.close()

111 to 330, 341 to 975


In [64]:
### Remove small tumor blobs from probability mask ###
from scipy import ndimage as ndi

def remove_small_tumor_blobs(prob_mask, min_region_tiles=5, prob_thresh=0.5):
    """
    prob_mask        : 2D array of probabilities (0–1)
    min_region_tiles : minimum blob size (in tiles)
    prob_thresh      : probability threshold to define 'tumor' tiles
    """
    # 1) Binary tumor mask
    det = prob_mask >= prob_thresh

    # 2) Connected components (8-connectivity)
    labels, num = ndi.label(det.astype(bool), structure=np.ones((3, 3), dtype=int))
    if num == 0:
        # no blobs at all
        return prob_mask * 0.0

    # 3) Component sizes
    sizes = ndi.sum(det, labels, index=np.arange(1, num + 1)).astype(int)
    max_size = sizes.max()

    if max_size <= min_region_tiles:
        # All blobs are small → keep ONLY the largest
        largest_label = int(np.argmax(sizes)) + 1
        keep_mask = (labels == largest_label)
    else:
        # Keep blobs larger than threshold
        keep_labels = 1 + np.where(sizes > min_region_tiles)[0]  # convert idx→label
        keep_mask = np.isin(labels, keep_labels)

    # 4) Zero out probabilities outside kept blobs
    cleaned = prob_mask.copy()
    cleaned[~keep_mask] = 0.0
    return cleaned

In [65]:
### Get tissue thumbnail with GT overlay

roi_thumb = (xmin, ymin, xmax, ymax)
img_roi, img_roi_with_gt, offset, bbox_used = get_thumbnail_white_bg(
    slide_name=slide_name,
    gt_color=(0,0,255),
    pad_pixels=0,
    roi_bbox_thumb=roi_thumb
)

fig, ax = plt.subplots(figsize=(12, 12), dpi=150)
ax.imshow(
    img_roi_with_gt,
    extent=[xmin, xmax, ymax, ymin],
    origin='upper', interpolation='none'
)

plt.savefig(output_path + f'{pid}_{img_id}_thumbnail_with_gt.png', bbox_inches='tight')
plt.close() # avoid displaying in notebook, which could crash it

In [75]:
### Overlay cleaned PFMap++ mask on thumbnail with GT ###

fig, ax = plt.subplots(figsize=(12, 12), dpi=300)
plt.imshow(
    img_roi_with_gt,
    extent=[xmin, xmax, ymax, ymin],
    origin='upper', interpolation='none'
)

# set tumor threshold
threshold = 0.5

# get binary tumor mask
tumor_mask = (upscaled > threshold).astype(float)   # 1 = tumor, 0 = normal

# Make tumor=black by using a colormap with value=1 → black
cmap = colors.ListedColormap(['none', 'blue'])   # 0 → transparent, 1 → black

plt.imshow(
    tumor_mask,
    cmap=cmap,
    interpolation='none', 
    alpha=0.2
)

plt.xlim(xmin, xmax)
plt.ylim(ymax, ymin)

plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel(f'X (pixels/{down_scale})', fontsize=16)
plt.ylabel(f'Y (pixels/{down_scale})', fontsize=16)

plt.xticks(range(1500, 3500 + 1, 500))

# Add legend
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='none', edgecolor='red', lw=1.5, label='Ground truth'),
    Patch(facecolor='blue', edgecolor='blue', alpha=0.25, lw=1.5, label='HaMap++ mask'),
]

plt.legend(
    handles=legend_elements,
    loc='best',
    frameon=True,
    fontsize=16,         # text size
    handlelength=1.9,    # line handle length
    handleheight=1.3,    # vertical handle height
    borderpad=0.6,       # padding inside legend box
    labelspacing=0.5     # spacing between entries
)

plt.savefig(output_path + f'{slide_name}_thumbnail_with_gt.png', bbox_inches='tight')
plt.close()

In [76]:
### Draw original PFMap on top of GT and slide thumbnail ###

mask_name = os.path.join(pfmap_path, 'C237_P5_0.5.npy')

mask_gaze = np.load(mask_name)

scale = int(32 / down_scale)
mask_pfmap = np.repeat(
    np.repeat(mask_gaze, scale, axis=0), 
    scale, axis=1
)

# Make tumor=black by using a colormap with value=1 → black
cmap = colors.ListedColormap(['none', 'blue'])   # 0 → transparent, 1 → black

# plot iamge with gt first
fig, ax = plt.subplots(figsize=(12, 12), dpi=300)
plt.imshow(
    img_roi_with_gt,
    extent=[xmin, xmax, ymax, ymin],
    origin='upper', interpolation='none'
)

# plot pfmap overlay
plt.imshow(
    mask_pfmap,
    cmap=cmap,
    interpolation='none', 
    alpha=0.3
)

plt.xlim(xmin, xmax)
plt.ylim(ymax, ymin)

plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel(f'X (pixels/{down_scale})', fontsize=16)
plt.ylabel(f'Y (pixels/{down_scale})', fontsize=16)

plt.xticks(range(1500, 3500 + 1, 500))

# Add legend
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='none', edgecolor='red', lw=1.5, label='Ground truth'),
    Patch(facecolor='blue', edgecolor='blue', alpha=0.3, lw=1.5, label='HaMap mask'),
]

plt.legend(
    handles=legend_elements,
    loc='best',
    frameon=True,
    fontsize=16,         # text size
    handlelength=1.9,    # line handle length
    handleheight=1.3,    # vertical handle height
    borderpad=0.6,       # padding inside legend box
    labelspacing=0.5     # spacing between entries
)

plt.savefig(output_path + f'{slide_name}_PFMap_with_gt.png', bbox_inches='tight')
plt.close()

In [11]:
### draw lesion annotations on black background ###

lesion_mask_file = os.path.join(lesion_mask_path, f"{slide_name}.xml")

fig, ax = draw_lesions_on_black(lesion_mask_file, down_scale, xmin, ymin, xmax, ymax,
                          figsize=(6, 6), linewidth=1.5)

# set tumor threshold
threshold = 0.5

# clean the mask
cleaned_mask = remove_small_tumor_blobs(upscaled, min_region_tiles=0, prob_thresh=threshold)

# Make tumor=black by using a colormap with value=1 → black
cmap = colors.ListedColormap(['none', 'white'])   # 0 → transparent, 1 → black

ax.imshow(
    cleaned_mask,
    cmap=cmap,
    interpolation='none', 
    alpha=1
)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymax, ymin)

fig.savefig(output_path + f'{slide_name}_expanded_mask_on_gt.png', bbox_inches='tight')
plt.close()